## Hyperparameter optimization

hyper-parameters, i.e. the parameters that control the training/fitting process of the model.

The model has many right answers. So, how would you find the best parameters? 
- A method would be to evaluate all the combinations and see which one improves the metric. Let’s see how this is done.

In [1]:
best_accuracy = 0
best_parameters = {"a": 0, "b": 0, "c": 0}

In [ ]:
# loop over all values for a, b & c
for a in range(1, 11):
 for b in range(1, 11):
    for c in range(1, 11):
        # inititalize model with current parameters
        model = MODEL(a, b, c)
        # fit the model
        model.fit(training_data)
        # make predictions
        preds = model.predict(validation_data)
        # calculate accuracy
        accuracy = metrics.accuracy_score(targets, preds)
        # save params if current accuracy
        # is greater than best accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_parameters["a"] = a
            best_parameters["b"] = b
            best_parameters["c"] = c


A search over the grid to find the best combination of parameters is known as `grid search.`

In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection

In [2]:
# read the training data
df = pd.read_csv(r"C:\Users\lenovo\Desktop\Disha Github\Machine Learning Approaches\Machine-Learning\datasets\train.csv")

In [3]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [4]:
# here we have training features
X = df.drop("price_range", axis=1).values
 # and the targets
y = df.price_range.values

In [5]:
 # define the model here
 # i am using random forest with n_jobs=-1
 # n_jobs=-1 => use all cores
classifier = ensemble.RandomForestClassifier(n_jobs=-1)

In [6]:
# define a grid of parameters
 # this can be a dictionary or a list of
 # dictionaries
param_grid = {
 "n_estimators": [100, 200, 250, 300, 400, 500],
 "max_depth": [1, 2, 5, 7, 11, 15],
 "criterion": ["gini", "entropy"]
 }

In [7]:
# initialize grid search
 # estimator is the model that we have defined
 # param_grid is the grid of parameters
 # we use accuracy as our metric. you can define your own
 # higher value of verbose implies a lot of details are printed
 # cv=5 means that we are using 5 fold cv (not stratified)
model = model_selection.GridSearchCV(
estimator=classifier, 
param_grid=param_grid, 
scoring="accuracy",
verbose=10, 
n_jobs=1,
cv=5
)

In [8]:
# fit the model and extract best score
model.fit(X, y)
print(f"Best score: {model.best_score_}")

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5; 1/72] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 1/5; 1/72] END criterion=gini, max_depth=1, n_estimators=100;, score=0.583 total time=   6.6s
[CV 2/5; 1/72] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 2/5; 1/72] END criterion=gini, max_depth=1, n_estimators=100;, score=0.532 total time=   0.1s
[CV 3/5; 1/72] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 3/5; 1/72] END criterion=gini, max_depth=1, n_estimators=100;, score=0.662 total time=   0.2s
[CV 4/5; 1/72] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 4/5; 1/72] END criterion=gini, max_depth=1, n_estimators=100;, score=0.613 total time=   0.2s
[CV 5/5; 1/72] START criterion=gini, max_depth=1, n_estimators=100..............
[CV 5/5; 1/72] END criterion=gini, max_depth=1, n_estimators=100;, score=0.547 total time=   0.1s
[CV 1/5; 2/72] START criterion=gini, max_de

In [9]:
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print(f"\t{param_name}: {best_parameters[param_name]}")

Best parameters set:
	criterion: entropy
	max_depth: 15
	n_estimators: 300


### Random search

- we randomly select a combination of parameters and calculate the cross-validation score.
- The time consumed here is less than grid search because we do not evaluate over all different combinations of parameters.

In [10]:
# define a grid of parameters
 # this can be a dictionary or a list of
 # dictionaries
param_grid_random = {
"n_estimators": np.arange(100, 1500, 100),
"max_depth": np.arange(1, 31),
"criterion": ["gini", "entropy"]
}

In [11]:

 # initialize random search
 # estimator is the model that we have defined
 # param_distributions is the grid/distribution of parameters
 # we use accuracy as our metric. you can define your own
 # higher value of verbose implies a lot of details are printed
 # cv=5 means that we are using 5 fold cv (not stratified)
 # n_iter is the number of iterations we want
 # if param_distributions has all the values as list,
 # random search will be done by sampling without replacement
 # if any of the parameters come from a distribution,
 # random search uses sampling with replacement
model_random = model_selection.RandomizedSearchCV(
 estimator=classifier, 
 param_distributions=param_grid_random,
 n_iter=20,
 scoring="accuracy",
 verbose=10, 
 n_jobs=1,
 cv=5
 )

In [12]:
# fit the model and extract best score
model_random.fit(X, y)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5; 1/20] START criterion=gini, max_depth=10, n_estimators=700.............
[CV 1/5; 1/20] END criterion=gini, max_depth=10, n_estimators=700;, score=0.875 total time=   7.6s
[CV 2/5; 1/20] START criterion=gini, max_depth=10, n_estimators=700.............
[CV 2/5; 1/20] END criterion=gini, max_depth=10, n_estimators=700;, score=0.887 total time=   1.5s
[CV 3/5; 1/20] START criterion=gini, max_depth=10, n_estimators=700.............
[CV 3/5; 1/20] END criterion=gini, max_depth=10, n_estimators=700;, score=0.895 total time=   1.0s
[CV 4/5; 1/20] START criterion=gini, max_depth=10, n_estimators=700.............
[CV 4/5; 1/20] END criterion=gini, max_depth=10, n_estimators=700;, score=0.870 total time=   1.2s
[CV 5/5; 1/20] START criterion=gini, max_depth=10, n_estimators=700.............
[CV 5/5; 1/20] END criterion=gini, max_depth=10, n_estimators=700;, score=0.860 total time=   1.1s
[CV 1/5; 2/20] START criterion=gini, m

In [15]:
print(f"Best score: {model_random.best_score_}")
print("Best parameters set:")
best_parameters = model_random.best_estimator_.get_params()
for param_name in sorted(param_grid_random.keys()):
 print(f"\t{param_name}: {best_parameters[param_name]}")

Best score: 0.8869999999999999
Best parameters set:
	criterion: entropy
	max_depth: 30
	n_estimators: 1400
